#Import library

In [1]:
## single frame analysis: whether there are large spatta?
import cv2
from IPython import display
from matplotlib import pyplot as plt
import time
from numpy import dtype,uint8
import scipy.ndimage as ndimage
import glob
import pandas as pd
import numpy as np
import os

#Check number of images

#20220401

In [2]:
dir = r'C:\Users\Furukawa\Documents\TechAssistant\dataset\imgs\20220401'
def checkNum(dir=dir,date_20221020=False):
    for i in range(1,13):
        for j in range(1,3):
            if date_20221020 :
                src_img_dir = os.path.join(dir,"{}\crop".format((i-1)*2+j+20))
            else:
                #get num of file in a directory
                src_img_dir = os.path.join(dir,"{}_{}\crop".format(i,j))

            num_photo = sum(os.path.isfile(os.path.join(src_img_dir,name)) for name in os.listdir(src_img_dir))

            print(f"({i},{j})::",num_photo)

checkNum()

(1,1):: 540
(1,2):: 537
(2,1):: 1080
(2,2):: 1079
(3,1):: 2162
(3,2):: 2162
(4,1):: 540
(4,2):: 540
(5,1):: 1080
(5,2):: 1081
(6,1):: 2163
(6,2):: 2162
(7,1):: 540
(7,2):: 540
(8,1):: 1082
(8,2):: 1079
(9,1):: 2161
(9,2):: 2162
(10,1):: 540
(10,2):: 540
(11,1):: 1080
(11,2):: 1080
(12,1):: 2161
(12,2):: 2092


#20220603

In [3]:
DIR_20220603 = r'C:\Users\Furukawa\Documents\TechAssistant\dataset\imgs\20220603'
checkNum(dir = DIR_20220603)

(1,1):: 540
(1,2):: 533
(2,1):: 1079
(2,2):: 1080
(3,1):: 2162
(3,2):: 2163
(4,1):: 457
(4,2):: 540
(5,1):: 1079
(5,2):: 1080
(6,1):: 2162
(6,2):: 2162
(7,1):: 541
(7,2):: 540
(8,1):: 1080
(8,2):: 1080
(9,1):: 2162
(9,2):: 2163
(10,1):: 539
(10,2):: 539
(11,1):: 1080
(11,2):: 1079
(12,1):: 2162
(12,2):: 2162


#20221020

In [4]:
DIR_20221020 = r'C:\Users\Furukawa\Documents\TechAssistant\dataset\imgs\20221020'
checkNum(dir = DIR_20221020,date_20221020=True)

(1,1):: 244
(1,2):: 448
(2,1):: 895
(2,2):: 245
(3,1):: 448
(3,2):: 863
(4,1):: 235
(4,2):: 430
(5,1):: 857
(5,2):: 233
(6,1):: 312
(6,2):: 880
(7,1):: 236
(7,2):: 434
(8,1):: 866
(8,2):: 240
(9,1):: 440
(9,2):: 883
(10,1):: 240
(10,2):: 442
(11,1):: 889
(11,2):: 242
(12,1):: 447
(12,2):: 892


#concatenate imgs

In [50]:
def concatImg(x_patch = 3, y_patch = 3,INVERSE = True):
    h,w,c = 80,80,1
    H = h*3
    W = w*3
    concat = np.zeros((H,W,c))
    folder_path = r'C:\Users\Furukawa\Documents\TechAssistant\dataset\imgs\20220401\1_1\crop'
    for i in range(9):
        file = "{:04d}.jpg".format(i+10)
        path = os.path.join(folder_path,file)
        img = cv2.imread(path)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        img = np.expand_dims(np.array(img).astype("int32"),2)
        #new to old
        if INVERSE:
            row =  (x_patch-1) - i//3
            column = (y_patch-1) - i%3
            concat[h*row:h*(row+1),w*column:w*(column+1),:] = img
        #old to new
        else:
            row = i//3
            column = i%3
            concat[h*row:h*(row+1),w*column:w*(column+1),:] = img
    cv2.imwrite(r'C:\Users\Furukawa\Documents\TechAssistant\dataset\imgs\20220401\concat_inverse.jpg',concat)

concatImg(INVERSE=True)

In [39]:
import numpy as np
import os
import random

class Dataset():
  def __init__(self,start_index=13,x_patch = 3, y_patch = 3, height = 80,width = 80,speed=100,
                date = "20220401",
                src_dir = r'C:\Users\Furukawa\Documents\TechAssistant\dataset\imgs\20220401',
                dir_concatImg = "concat_MAE_spatterframe_estimation"
              ):
    self.start_index=start_index
    #num of patches
    self.x_patch = x_patch
    self.y_patch = y_patch  
    #size for 1 patch
    self.height = height
    self.width = width
    #size for concatenated img
    self.H = x_patch*height
    self.W = y_patch*width
    #date of dataset
    self.date = date
    #img src directory
    self.src_dir = src_dir
    #directory for concat img
    self.dir_concatImg = dir_concatImg
    self.speed=speed
    #load spatter labels from .txt file
    if speed==100:
        dir = r'C:\Users\Furukawa\Documents\TechAssistant\Video\20230823_synchronized\IR_1000W, B_500W, {speed}mm_s\video'.format(speed=speed)
        self.spatter_data_13 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(13)), dtype='int').tolist()
        self.spatter_data_14 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(14)), dtype='int').tolist()
        self.spatter_data_15 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(15)), dtype='int').tolist()
        self.spatter_data_16 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(16)), dtype='int').tolist()
        #self.spatter_data_17 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(17)), dtype='int').tolist()
        self.spatter_data_18 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(18)), dtype='int').tolist()
        self.spatter_data_19 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(19)), dtype='int').tolist()
        self.spatter_data_20 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(20)), dtype='int').tolist()
        self.spatter_data_21 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(21)), dtype='int').tolist()
        self.spatter_data_22 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(22)), dtype='int').tolist()
        self.spatter_data_23 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(23)), dtype='int').tolist()
        self.spatter_data_24 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(24)), dtype='int').tolist()
        self.spatter_data_25 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(25)), dtype='int').tolist()
        self.spatter_data_26 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(26)), dtype='int').tolist()
        self.spatter_data_27 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(27)), dtype='int').tolist()
        self.spatter_data_28 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(28)), dtype='int').tolist()
        self.spatter_data_29 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(29)), dtype='int').tolist()
        self.spatter_data_30 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(30)), dtype='int').tolist()
        self.spatter_data_31 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(31)), dtype='int').tolist()
        self.spatter_data_32 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(32)), dtype='int').tolist()
        self.spatter_data_33 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(33)), dtype='int').tolist()
        self.spatter_data_34 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(34)), dtype='int').tolist()
        self.spatter_data_35 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(35)), dtype='int').tolist()
        self.spatter_data_36 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(36)), dtype='int').tolist()
        self.spatter_data_37 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(37)), dtype='int').tolist()
        self.spatter_data_38 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(38)), dtype='int').tolist()
        self.spatter_data_39 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(39)), dtype='int').tolist()
        self.spatter_data_40 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(40)), dtype='int').tolist()
        self.spatter_data_41 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(41)), dtype='int').tolist()
        self.spatter_data_42 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(42)), dtype='int').tolist()
    elif speed==200:
      dir = r'C:\Users\Furukawa\Documents\TechAssistant\Video\20230823_synchronized\IR_1000W, B_500W, {speed}mm_s\video'.format(speed=speed)
      self.spatter_data_51 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(51)), dtype='int').tolist()
      self.spatter_data_52 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(52)), dtype='int').tolist()
      self.spatter_data_53 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(53)), dtype='int').tolist()
      self.spatter_data_54 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(54)), dtype='int').tolist()
      self.spatter_data_55 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(55)), dtype='int').tolist()
      self.spatter_data_56 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(56)), dtype='int').tolist()
      self.spatter_data_57 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(57)), dtype='int').tolist()
      self.spatter_data_58 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(58)), dtype='int').tolist()
      self.spatter_data_59 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(59)), dtype='int').tolist()
      self.spatter_data_60 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(60)), dtype='int').tolist()
      self.spatter_data_61 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(61)), dtype='int').tolist()
      self.spatter_data_62 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(62)), dtype='int').tolist()
      self.spatter_data_63 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(63)), dtype='int').tolist()
      self.spatter_data_64 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(64)), dtype='int').tolist()
      self.spatter_data_65 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(65)), dtype='int').tolist()
      self.spatter_data_66 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(66)), dtype='int').tolist()
      self.spatter_data_67 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(67)), dtype='int').tolist()
      self.spatter_data_68 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(68)), dtype='int').tolist()
      self.spatter_data_69 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(69)), dtype='int').tolist()
      self.spatter_data_70 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(70)), dtype='int').tolist()
      self.spatter_data_71 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(71)), dtype='int').tolist()
      self.spatter_data_72 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(72)), dtype='int').tolist()
      self.spatter_data_73 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(73)), dtype='int').tolist()
      self.spatter_data_74 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(74)), dtype='int').tolist()
      self.spatter_data_75 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(75)), dtype='int').tolist()
      self.spatter_data_76 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(76)), dtype='int').tolist()
      self.spatter_data_77 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(77)), dtype='int').tolist()
      self.spatter_data_78 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(78)), dtype='int').tolist()
      self.spatter_data_79 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(79)), dtype='int').tolist()
      self.spatter_data_80 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(80)), dtype='int').tolist()


  def concatImg(self,index,src_img_dir,save_dir,INVERSE = False):
      """concatenate images
      Args:
        index: index for spatter frame, so index is the latest index in 9 frames, ie. index, index-1,index-2,..., index-8
        src_img_dir : source directory of cropped imgs
        save_dir : save dir for concatenate images
        x_patch, y_patch : number of patches in x and y axis
        INVERSE : patch order. If INVERSE is True, new to old is applied
      Return:
        file_path : path for concat img
        write concatenated images into designated directory
      """
      concat = np.zeros((self.H,self.W,1))
      for i in range(9):
          file = "{:03d}.jpg".format(index-i) #from new to old
          path = os.path.join(src_img_dir,file)
          img = cv2.imread(path)
          img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #gray scale
          img = np.expand_dims(np.array(img).astype("int32"),2) #add color channel : (height,width,1)
          #new to old
          if INVERSE:
              row = i//3
              column = i%3
              concat[self.height*row:self.height*(row+1),self.width*column:self.width*(column+1),:] = img
          #old to new
          else:
              row =  (self.x_patch-1) - i//3
              column = (self.y_patch-1) - i%3
              concat[self.height*row:self.height*(row+1),self.width*column:self.width*(column+1),:] = img
      file_path = os.path.join(save_dir,f"{index:03d}.jpg")
      cv2.imwrite(file_path,concat)
      return file_path

  # adjust the number of no spatter below 51
  def make(self,filename,filename_test,div,sec):
    #フォルダに存在するファイルを取得する、0001.jpg~0005.jpg→{}/*.jpg
    
    if self.speed==100:
       spatter = [self.spatter_data_13,self.spatter_data_14,self.spatter_data_15,
                  self.spatter_data_16,[],self.spatter_data_18,self.spatter_data_19,
                  [self.spatter_data_20],self.spatter_data_21,[self.spatter_data_22],
                  self.spatter_data_23,self.spatter_data_24,self.spatter_data_25,
                  self.spatter_data_26,self.spatter_data_27,[self.spatter_data_28],
                  self.spatter_data_29,self.spatter_data_30,self.spatter_data_31,
                  self.spatter_data_32,self.spatter_data_33,self.spatter_data_34,
                  self.spatter_data_35,[self.spatter_data_36],self.spatter_data_37,
                  self.spatter_data_38,self.spatter_data_39,[self.spatter_data_40],
                  [self.spatter_data_41],self.spatter_data_42
                  ]
    elif self.speed == 200:
      spatter = [[self.spatter_data_51],self.spatter_data_52,[self.spatter_data_53],
                  self.spatter_data_54,self.spatter_data_55,self.spatter_data_56,
                  [self.spatter_data_57],self.spatter_data_58,[self.spatter_data_59],
                  self.spatter_data_60,[self.spatter_data_61],self.spatter_data_62,
                  self.spatter_data_63,self.spatter_data_64,self.spatter_data_65,
                  [self.spatter_data_66],self.spatter_data_67,self.spatter_data_68,
                  [self.spatter_data_69],self.spatter_data_70,self.spatter_data_71,
                  self.spatter_data_72,[self.spatter_data_73],[self.spatter_data_74],
                  self.spatter_data_75,self.spatter_data_76,[self.spatter_data_77],
                  [self.spatter_data_78],self.spatter_data_79,[self.spatter_data_80]
                  ]
       
    #set src img directory
    src_img_dir = os.path.join(self.src_dir,"{}\crop".format(div))
    #make save directory
    save_dir= os.path.join(self.src_dir,"{}\{}".format(div,self.dir_concatImg))
    if not os.path.exists(save_dir):
      os.mkdir(save_dir)

    #spatter count
    count_spatter = 0
    #get num of file in a directory
    num_photo = sum(os.path.isfile(os.path.join(src_img_dir,name)) for name in os.listdir(src_img_dir))
    print("{}-{} progressing :: {}".format(div,sec,num_photo))
    #make dataset
    for i in range(10,num_photo-10):
      #if last frame is spatter and others are not spatter
      if ((i in spatter[div-self.start_index]) and 
          ((i-1) not in spatter[div-self.start_index]) and ((i-2) not in spatter[div-self.start_index]) and 
          ((i-3) not in spatter[div-self.start_index]) and ((i-4) not in spatter[div-self.start_index]) and ((i-5) not in spatter[div-self.start_index]) and 
          ((i-6) not in spatter[div-self.start_index]) and ((i-7) not in spatter[div-self.start_index]) and ((i-8) not in spatter[div-self.start_index])
        ):
          #concatenate images
          file_path = self.concatImg(index=i,src_img_dir=src_img_dir,save_dir=save_dir,INVERSE = False)
          if self.speed==100:
             threshold_train=0.12
          elif self.speed==200:
             threshold_train=0.1
          if random.random()>threshold_train:#train data
            with open(filename,"a",newline='') as f:
              writer = csv.writer(f)
              writer.writerow([1,file_path])
          else:
            with open(filename_test,"a",newline='') as f:
              writer = csv.writer(f)
              writer.writerow([1,file_path])
          count_spatter += 1
          print(f"spatter detected : {count_spatter}")

#20220401

#Initialize dataset file

In [40]:
import csv







#Initialize csv dataset?
INITIALIZE = True







#prepare csv files
file_dir = r'C:\Users\Furukawa\Documents\TechAssistant\dataset\csv\ViT\MAE_spatterframe_estimation'
filename=os.path.join(file_dir,"train_IR1000_B500_S100.csv")
filename_test=os.path.join(file_dir,"test_IR1000_B500_S100.csv")
#///////////////////
if INITIALIZE:
  #make csv file
  with open(filename,"w") as f:
    writer = csv.writer(f)
    writer.writerow(["Spatter","Path"])

  with open(filename_test,"w") as f:
    writer = csv.writer(f)
    writer.writerow(["Spatter","Path"])

In [41]:
START=13
dataset = Dataset(start_index=START,x_patch = 3, y_patch = 3, height = 80,width = 80,speed=100,
                date = "20230823",
                src_dir = r'C:\Users\Furukawa\Documents\TechAssistant\dataset\imgs\20230823\IR1000_B500_S100',
                dir_concatImg = "concat_MAE_spatterframe_estimation"
                )

#make dataset one by one
for i in range(13,43):
  for j in range(1,2):
    dataset.make(filename=filename,filename_test=filename_test,div = i,sec = j)

C:\Users\Furukawa\AppData\Local\Temp\ipykernel_12892\3656635627.py:31: UserWarning: loadtxt: input contained no data: "C:\Users\Furukawa\Documents\TechAssistant\Video\20230823_synchronized\IR_1000W, B_500W, 100mm_s\video\13_spatter.txt"
  self.spatter_data_13 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(13)), dtype='int').tolist()
C:\Users\Furukawa\AppData\Local\Temp\ipykernel_12892\3656635627.py:33: UserWarning: loadtxt: input contained no data: "C:\Users\Furukawa\Documents\TechAssistant\Video\20230823_synchronized\IR_1000W, B_500W, 100mm_s\video\15_spatter.txt"
  self.spatter_data_15 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(15)), dtype='int').tolist()
C:\Users\Furukawa\AppData\Local\Temp\ipykernel_12892\3656635627.py:34: UserWarning: loadtxt: input contained no data: "C:\Users\Furukawa\Documents\TechAssistant\Video\20230823_synchronized\IR_1000W, B_500W, 100mm_s\video\16_spatter.txt"
  self.spatter_data_16 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(16

13-1 progressing :: 499
14-1 progressing :: 498
spatter detected : 1
spatter detected : 2
15-1 progressing :: 498
16-1 progressing :: 498
17-1 progressing :: 499
18-1 progressing :: 498
19-1 progressing :: 497
20-1 progressing :: 498
21-1 progressing :: 498
22-1 progressing :: 498
23-1 progressing :: 497
24-1 progressing :: 497
spatter detected : 1
25-1 progressing :: 498
26-1 progressing :: 495
27-1 progressing :: 495
spatter detected : 1
spatter detected : 2
spatter detected : 3
28-1 progressing :: 498
29-1 progressing :: 495
spatter detected : 1
spatter detected : 2
30-1 progressing :: 497
31-1 progressing :: 497
spatter detected : 1
spatter detected : 2
spatter detected : 3
spatter detected : 4
spatter detected : 5
32-1 progressing :: 497
33-1 progressing :: 498
spatter detected : 1
34-1 progressing :: 498
35-1 progressing :: 499
spatter detected : 1
spatter detected : 2
36-1 progressing :: 497
spatter detected : 1
37-1 progressing :: 499
38-1 progressing :: 492
spatter detected : 

## check dataset

In [42]:
import pandas as pd
file_train = pd.read_csv(filename)
train = file_train.values
labels_train = train[:,0]
print(train.shape)
file_test = pd.read_csv(filename_test)
test = file_test.values
labels_test = test[:,0]
print(test.shape)
def count_spatter(labels):
    count = 0
    for i in range(labels.shape[0]):
        if labels[i] == 1:
            count += 1
    print(f"Spatter/toal : {count}/{labels.shape[0]}")

print("train data")
count_spatter(labels_train)
print("test data")
count_spatter(labels_test)

(20, 2)
(2, 2)
train data
Spatter/toal : 20/20
test data
Spatter/toal : 2/2


# speed=200 mm/sec

In [43]:
import csv







#Initialize csv dataset?
INITIALIZE = True







#prepare csv files
file_dir = r'C:\Users\Furukawa\Documents\TechAssistant\dataset\csv\ViT\MAE_spatterframe_estimation'
filename=os.path.join(file_dir,"train_IR1000_B500_S200.csv")
filename_test=os.path.join(file_dir,"test_IR1000_B500_S200.csv")
#///////////////////
if INITIALIZE:
  #make csv file
  with open(filename,"w") as f:
    writer = csv.writer(f)
    writer.writerow(["Spatter","Path"])

  with open(filename_test,"w") as f:
    writer = csv.writer(f)
    writer.writerow(["Spatter","Path"])

In [44]:
START=51
dataset = Dataset(start_index=START,x_patch = 3, y_patch = 3, height = 80,width = 80,speed=200,
                date = "20230823",
                src_dir = r'C:\Users\Furukawa\Documents\TechAssistant\dataset\imgs\20230823\IR1000_B500_S200',
                dir_concatImg = "concat_MAE_spatterframe_estimation"
                )

#make dataset one by one
for i in range(51,81):
  for j in range(1,2):
    dataset.make(filename=filename,filename_test=filename_test,div = i,sec = j)

C:\Users\Furukawa\AppData\Local\Temp\ipykernel_12892\3656635627.py:66: UserWarning: loadtxt: input contained no data: "C:\Users\Furukawa\Documents\TechAssistant\Video\20230823_synchronized\IR_1000W, B_500W, 200mm_s\video\54_spatter.txt"
  self.spatter_data_54 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(54)), dtype='int').tolist()
C:\Users\Furukawa\AppData\Local\Temp\ipykernel_12892\3656635627.py:67: UserWarning: loadtxt: input contained no data: "C:\Users\Furukawa\Documents\TechAssistant\Video\20230823_synchronized\IR_1000W, B_500W, 200mm_s\video\55_spatter.txt"
  self.spatter_data_55 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(55)), dtype='int').tolist()
C:\Users\Furukawa\AppData\Local\Temp\ipykernel_12892\3656635627.py:70: UserWarning: loadtxt: input contained no data: "C:\Users\Furukawa\Documents\TechAssistant\Video\20230823_synchronized\IR_1000W, B_500W, 200mm_s\video\58_spatter.txt"
  self.spatter_data_58 = np.loadtxt(os.path.join(dir,'{}_spatter.txt'.format(58

51-1 progressing :: 499
spatter detected : 1
52-1 progressing :: 497
spatter detected : 1
spatter detected : 2
53-1 progressing :: 499
54-1 progressing :: 499
55-1 progressing :: 498
56-1 progressing :: 498
spatter detected : 1
spatter detected : 2
spatter detected : 3
spatter detected : 4
spatter detected : 5
spatter detected : 6
spatter detected : 7
spatter detected : 8
57-1 progressing :: 499
58-1 progressing :: 498
59-1 progressing :: 499
spatter detected : 1
60-1 progressing :: 498
61-1 progressing :: 498
spatter detected : 1
62-1 progressing :: 496
63-1 progressing :: 496
spatter detected : 1
spatter detected : 2
spatter detected : 3
spatter detected : 4
spatter detected : 5
spatter detected : 6
spatter detected : 7
spatter detected : 8
spatter detected : 9
64-1 progressing :: 497
65-1 progressing :: 497
66-1 progressing :: 498
67-1 progressing :: 499
68-1 progressing :: 499
69-1 progressing :: 498
70-1 progressing :: 499
71-1 progressing :: 499
72-1 progressing :: 499
73-1 progr

In [45]:
import pandas as pd
file_train = pd.read_csv(filename)
train = file_train.values
labels_train = train[:,0]
print(train.shape)
file_test = pd.read_csv(filename_test)
test = file_test.values
labels_test = test[:,0]
print(test.shape)
def count_spatter(labels):
    count = 0
    for i in range(labels.shape[0]):
        if labels[i] == 1:
            count += 1
    print(f"Spatter/toal : {count}/{labels.shape[0]}")

print("train data")
count_spatter(labels_train)
print("test data")
count_spatter(labels_test)

(24, 2)
(2, 2)
train data
Spatter/toal : 24/24
test data
Spatter/toal : 2/2
